In [ ]:
from tardis.io.atom_data import download_atom_data
from tardis import run_tardis
from tardis.io.atom_data.base import AtomData
from tardis.simulation import Simulation
import matplotlib.pyplot as plt
from tardis.visualization import SDECPlotter
import matplotlib
import numpy as np
import pandas as pd
import os
import re
import csv
from astropy import units as u
from astropy.cosmology import Planck18 as cosmo
from scipy.signal import find_peaks
import periodictable
download_atom_data('kurucz_cd23_chianti_H_He')

# Running the Simulation

In [ ]:
# Labels of 03fg-like SNe

hvf1, hvf2, dn12, dc09, aa11 = 'SN2020hvf (-10.4)', 'SN2020hvf (-3.3)', 'SN2012dn (-2.3)', 'SN2009dc (+5.8)', 'SN2011aa (+8.1)'

z_hvf, z_dn12, z_dc09, z_aa11 = 0.0057, 0.0102, 0.0214, 0.0124    # <------------------------------------  redshift

#------------ Information of the Object -----------------------------

obj = hvf2
identifier="2020hvf_2"
filename = 'sn2020hvf_2.flm'
z = z_hvf

#----------------------------------------------------------------------------------

d_mpc = cosmo.comoving_distance(z).value
d_cm = (d_mpc * u.Mpc).to(u.cm)
d_term = (4 * np.pi * d_cm**2)

sim = run_tardis(f"{identifier}.yml", virtual_packet_logging=True)


matplotlib.rc('xtick', labelsize=28)
matplotlib.rc('ytick', labelsize=28)
plt.rc('font', size=28)
plt.figure(figsize=(16, 12))

#----------------------------------------

spectrum = np.loadtxt(filename)

obj_wave = spectrum[:, 0] 
obj_flux = spectrum[:, 1]  

wavelength = sim.spectrum_solver.spectrum_integrated.wavelength.to('angstrom').value
lum_density = sim.spectrum_solver.spectrum_integrated.luminosity_density_lambda.value

model_wave = np.array(wavelength)
model_lum_density = np.array(lum_density)
model_flux = model_lum_density / d_term

# Plotting UV + Optical -----------------------

plt.xlabel('Rest wavelength ($\AA$)')
plt.ylabel('$F_{\\lambda}$ [erg $\\mathrm{s^{-1}}$ $\\mathrm{cm^{-2}}$ $\\mathrm{\\AA^{-1}}$]')
plt.xlim(2500, 8000)
plt.ylim(0, 1.6e-14)
plt.plot(obj_wave,obj_flux,label=obj, color='blue', linewidth=3)
plt.plot(model_wave,3.2*model_flux,label='TARDIS Spectrum', color='red', linewidth=3)
plt.legend()
plt.tight_layout()
plt.title("UV + Optical")
plt.savefig(f"{identifier}_opt.pdf")
plt.show()

#-------- Plotting the UV region -----------------

plt.figure(figsize=(16, 12))

plt.xlabel('Rest wavelength ($\AA$)')
plt.ylabel('$F_{\\lambda}$ [erg $\\mathrm{s^{-1}}$ $\\mathrm{cm^{-2}}$ $\\mathrm{\\AA^{-1}}$]')
plt.xlim(2500, 4500)
plt.ylim(0, 1.6e-14)
plt.plot(obj_wave,obj_flux,label=obj, color='blue', linewidth=3)
plt.plot(model_wave,3.2*model_flux,label='TARDIS Spectrum', color='red', linewidth=3)
plt.legend()
plt.tight_layout()
plt.title("UV")
plt.savefig(f"{identifier}_uv.pdf")
plt.show()

# SED plot UV

In [ ]:
spectrum = np.loadtxt(filename)

obj_mask = (spectrum[:, 0] >= 2500) & (spectrum[:, 0] <= 4500)
obj_wave = spectrum[obj_mask, 0]
obj_flux = spectrum[obj_mask, 1]


matplotlib.rc('xtick', labelsize=44)
matplotlib.rc('ytick', labelsize=44)
figsize = (28, 20)

plotter = SDECPlotter.from_simulation(sim)
observed_spectrum_wavelength = obj_wave * u.AA 
observed_spectrum_flux = 0.3*obj_flux * u.erg / (u.s * u.cm**2 * u.AA)

plotter.generate_plot_mpl(observed_spectrum = (observed_spectrum_wavelength, observed_spectrum_flux), 
                          distance = d_mpc * u.Mpc,blackbody_photosphere=False,cmapname='Paired',
                          packet_wvl_range=[2500, 4500] * u.AA,nelements=12,figsize=figsize)


observed_line = plt.gca().get_lines()[-1]
observed_line.set_linewidth(3)
observed_line.set_label(obj)

cbar = plt.gcf().axes[-1]
cbar.tick_params(labelsize=56)  

plt.legend(fontsize=38,loc='lower right', frameon=False)
plt.xlabel('Rest wavelength ($\AA$)',fontsize=46)
plt.ylabel('$F_{\\lambda}$ [erg $\\mathrm{s^{-1}}$ $\\mathrm{cm^{-2}}$ $\\mathrm{\\AA^{-1}}$]',fontsize=46)
plt.tight_layout()
plt.savefig(f"sed_{identifier}.pdf")

# SED Plot Optical

In [ ]:
spectrum = np.loadtxt(filename)

obj_mask2 = (spectrum[:, 0] >= 4000) & (spectrum[:, 0] <= 8000)
obj_wave = spectrum[obj_mask2, 0]
obj_flux = spectrum[obj_mask2, 1]

figsize = (16, 10)
plotter = SDECPlotter.from_simulation(sim)
observed_spectrum_wavelength = obj_wave * u.AA 
observed_spectrum_flux = 0.3*obj_flux * u.erg / (u.s * u.cm**2 * u.AA)


plotter.generate_plot_mpl( distance = d_mpc * u.Mpc,blackbody_photosphere=False,cmapname='Paired',
                          packet_wvl_range=[4000, 8000] * u.AA,figsize=figsize)
                     


plt.legend(fontsize=12,loc='upper right')

plt.savefig(f"sed_{identifier}_optical.pdf")